In [ ]:
#book flight reservations from one place to another using ai loop, it will loop till all the destinations are booked

In [5]:

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [8]:
openai=OpenAI()

In [7]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [ ]:
create_flight_reservation = {
    "name": "create_flight_reservation_from_one_place_to_another",
    "description": "Create a flight reservation from one place to another",
    "parameters": {
        "type": "object",
        "properties": {
            "flight_number": {
                "type": "string",
                "description": "The flight number of the reservation"
            },
            "origin": {
                "type": "string",
                "description": "The origin of the flight reservation"
            },
            "destination": {
                "type": "string",
                "description": "The destination of the flight reservation"
            },
            "date": {
                "type": "string",
                "description": "The date of the flight reservation"
            }
        },  
        "required": ["flight_number", "origin", "destination", "date"]
    }
}

In [37]:
flight_available_api = {
    "name": "available_flight_from_one_place_to_another",
    "description": "Check flight availability from one place to another",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "The origin of the flight reservation"
            },
            "destination": {
                "type": "string",
                "description": "The destination of the flight reservation"
            },
            "date": {
                "type": "string",
                "description": "The date of the flight reservation in YYYY-MM-DD format"
            }
        },
        "required": ["origin", "destination", "date"]
    }
}   

In [36]:
def available_flight_from_one_place_to_another(origin, destination, date):
    # Here you would add the logic to check flight availability using an API or database

    flights = {
        ("London", "Paris"): [
        {"flight_number": "AA123", "origin": origin, "destination": destination, "date": date, "available_seats": 5, "price": 500},
        {"flight_number": "BB456", "origin": origin, "destination": destination, "date": date, "available_seats": 0, "price": 300},
        {"flight_number": "CC789", "origin": origin, "destination": destination, "date": date, "available_seats": 3, "price": 400}],
        ("New York", "London"): [
        {"flight_number": "DD123", "origin": origin, "destination": destination, "date": date, "available_seats": 2, "price": 700},
        {"flight_number": "EE456", "origin": origin, "destination": destination, "date": date, "available_seats": 0, "price": 600},
        {"flight_number": "FF789", "origin": origin, "destination": destination, "date": date, "available_seats": 4, "price": 800}]
    }

    return flights.get((origin, destination), [])

In [3]:
tools=[{"type": "function", "function": create_flight_reservation}]

In [4]:
def create_flight_reservation_from_one_place_to_another(origin, destination, date):
    # Here you would add the logic to create a flight reservation using an API or database
    print(f"Creating flight reservation from {origin} to {destination} on {date}")

In [33]:
def handle_too_calls(tool_calls):
    results=[]
    for call in tool_calls:
        if call.function.name == "create_flight_reservation_from_one_place_to_another":
            arguments = json.loads(call.function.arguments)
            result = create_flight_reservation_from_one_place_to_another(**arguments)
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": call.id})
    return results

In [19]:
def loop(messages):
    done=True
    while done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        if response.choices[0].message.tool_calls:
            tool_calls = response.choices[0].message.tool_calls
            tool_results = handle_too_calls(tool_calls)
            messages.append(response.choices[0].message)
            messages.extend(tool_results) 
        else:
            done=False
    show(response.choices[0].message.content)

In [9]:
system_message = """ You are a helpful travel assistant that can book flight reservations for users. You can create flight reservations from one place to another on a specific date. You will loop until all the destinations are booked. Use the tool provided to create flight reservations when needed.
"""

user_message = """ I want to book a flight from New York to London on 2024-07-01, and then from London to Paris on 2024-07-05."""

In [34]:
loop([{"role": "system", "content": system_message}, {"role": "user", "content": user_message}])

Creating flight reservation from New York to London on 2024-07-01
Creating flight reservation from London to Paris on 2024-07-05


Your flight reservations have been successfully booked:

1. From **New York** to **London** on **July 1, 2024**.
2. From **London** to **Paris** on **July 5, 2024**.

If you need any further assistance or additional bookings, feel free to ask!